In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.7MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

In [6]:
cd "/content/drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


In [7]:
ls

cifar10.ipynb                 day4_meta.ipynb  testNN.ipynb
ColabDriveGithub.ipynb        day5.ipynb       Untitled0.ipynb
czsowniki_nieregularne.ipynb  dw_matrix/       Untitled1.ipynb


In [8]:
cd dw_matrix

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv("data/men_shoes.csv", low_memory=False)

In [0]:
def run_model(feats, model=DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [17]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [18]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [20]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
def parse_features(x):
    output_dict = {}
    if str(x) == 'nan': return output_dict
    features = literal_eval(x.replace('\\"', '"'))
    for item in features:
      key = item['key'].lower().strip()
      value = item['value'][0].lower().strip()
      output_dict[key] = value
    return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [27]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [28]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df['features_parsed'].map(lambda feats: feats[key] if key in feats else np.nan)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0]*100

In [34]:
{k:v for k, v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [38]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand == df.feat_brand].shape

(8846, 531)

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]

df['colors_cat'] = df['colors'].map(lambda x: str(x).lower().strip()).factorize()[0]
df['ean_filled'] = df['ean'].fillna(-99)
df['manufacturer_cat'] = df['manufacturer'].map(lambda x: str(x).lower().strip()).factorize()[0]
df['feat_resizable_cat']

In [0]:
from itertools import combinations

In [0]:
feat_band width_cat, manufacturer_cat

In [462]:
results = list()
cs = list()
# for i in range(len(feats)):
#   print(i+1)
i = 1
for comb in combinations(feats, 6):
  print(i)
  i += 1
  model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=1)
  res = run_model(list(comb), model)
  results.append(res)
  cs.append(list(comb))

1
2
3
4
5
6
7


In [463]:
print(set(cs[2]))
print(set(feats))

{'feat_brand_cat', 'feat_material_cat', 'feat_sport_cat', 'dim3', 'brand_cat', 'dim1'}
{'feat_brand_cat', 'feat_color_cat', 'feat_material_cat', 'feat_sport_cat', 'dim3', 'brand_cat', 'dim1'}


In [464]:
# print(results[7], cs[7])
results

[(-56.93077175047498, 4.262702475772991),
 (-56.77627907587422, 3.9949231319194882),
 (-56.6888517371042, 4.1066774926707055),
 (-57.5193835499962, 4.689753090180989),
 (-56.624972390218396, 4.129897874325852),
 (-56.68247497313787, 4.121206803713881),
 (-59.75064530406621, 3.738292483362044)]

In [0]:
 feats = ['brand_cat', 'feat_brand_cat', 'feat_sport_cat', 'feat_material_cat', 'feat_color_cat', 'manufacturer_cat',
          'dim1', 'dim3']

#  'feat_band width_cat',, 'dim2', 'feat_resizable_cat', 'feat_age_cat', 'feat_number of wheels_cat', 'feat_condition_cat', 'feat_mpn_cat', 'weight_conv', 'feat_gender_cat', 'feat_style_cat',
# feats += nfeats
# feats = list(set(feats)), 'feat_look best on these face shapes:_cat', 
feats2 = ['brand_cat', 'feat_brand_cat', 'feat_sport_cat', 'feat_material_cat', 'feat_gender_cat', 'feat_mpn_cat', 'feat_condition_cat', 'feat_style_cat']
# for f in feats:
  # print(f, len(df[f].unique()))
# df['feat_look best on these face shapes:_cat'].unique()

In [478]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=1)
run_model(feats, model)

(-56.6217523353633, 4.286416060250029)

In [479]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=1)
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)


Weight,Feature
0.2667 ± 0.0107,brand_cat
0.1085 ± 0.0117,feat_material_cat
0.0617 ± 0.0045,manufacturer_cat
0.0241 ± 0.0025,feat_brand_cat
0.0134 ± 0.0025,dim3
0.0095 ± 0.0011,dim1
0.0083 ± 0.0015,feat_color_cat
0.0004 ± 0.0000,feat_sport_cat


In [45]:
df['brand'].value_counts(normalize=True)

nike                         0.097210
puma                         0.033315
ralph lauren                 0.028775
vans                         0.021116
new balance                  0.020295
                               ...   
rag & bone                   0.000055
foundry                      0.000055
spiewak                      0.000055
earrings-midwestjewellery    0.000055
aiden                        0.000055
Name: brand, Length: 1732, dtype: float64

In [0]:
for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]

In [0]:
feats += feats_cat
feats = list(set(feats))

In [275]:
list(df.columns)[-20:]

['feat_fabric care_cat',
 'feat_location - country_cat',
 'feat_hammer loop_cat',
 'feat_number of pieces_cat',
 'feat_cm_cat',
 'feat_design_cat',
 'feat_alarm_cat',
 'feat_domestic shipping_cat',
 'feat_athlete_cat',
 'feat_genre_cat',
 'feat_rise_cat',
 'feat_release_cat',
 'feat_type_cat',
 'dim1',
 'dim2',
 'dim3',
 'weight_conv',
 'colors_cat',
 'ean_filled',
 'manufacturer_cat']

In [0]:
nfeats = [c for c in list(df.columns) if c.find('_cat') != -1]

In [211]:
# df['dimension'].value_counts()
# n_feats = list(df.columns)[1:50]
df['feat_number of wheels'].value_counts()

0    7
Name: feat_number of wheels, dtype: int64

In [0]:
splitted = list(df['dimension'].fillna('nan').values)

In [0]:
# splitted = [s.split('in') if s != 'nan' else ['nan', 'nan', 'nan']  for s in splitted ]
a = list()
b = list()
c = list()
for s in splitted:
  a.append(float(s.split('in')[0].strip()) if s != 'nan' else -99)
  b.append(float(s.split('x ')[1].split(' in')[0].strip()) if s != 'nan' else -99)
  c.append(float(s.split('x ')[2].split(' in')[0].strip()) if s != 'nan' else -99)

In [0]:
df['dim1'] = a
df['dim2'] = b
df['dim3'] = c

In [122]:
df['weight'].unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [0]:
weights = {'lbs': 453.59, 'lb': 453.59, 'pounds': 453.59, 'ounces': 28.35, 'Kg': 1000, 'g': 1}

In [0]:
def convert_weight(x):
  if str(x) == 'nan': return -999
  else:
    w, u = str(x).split(' ')
    return float(w) * weights[u]
df['weight_conv'] = df['weight'].map(lambda x: convert_weight(x))

In [134]:
df['weight_conv'].unique()

array([-9.99000000e+02,  1.36077000e+03,  9.00000000e+00,  6.57705500e+02,
        2.04115500e+02,  4.53590000e+02,  1.04325700e+02,  2.26795000e+03,
        2.49474500e+03,  3.37924550e+03,  1.81436000e+03,  1.26864587e+03,
        1.76900100e+03,  2.08651400e+03,  9.52539000e+02,  5.01534463e+02,
        6.80385000e+03,  6.80400000e+01,  4.54000000e+02,  4.76269500e+01,
        2.57985000e+02,  2.17723200e+03,  2.76689900e+03,  2.94833500e+03,
        5.00808719e+02,  1.30000000e+03,  9.10000000e+01,  9.07180000e+03,
        2.72154000e+03,  3.86000000e+02,  3.67407900e+02,  2.04115500e+03,
        1.41750000e+01,  9.07180000e+02,  1.41973670e+03,  2.67618100e+03,
        2.78957850e+03,  8.84500500e+02,  9.75218500e+02,  1.40000000e+04,
        2.17178892e+02,  4.53590000e+03,  1.72364200e+02,  1.13397500e+03,
        3.12577941e+04,  4.50000000e+01,  5.93749310e+03,  9.52539000e+01,
        7.59763250e+03,  2.85761700e+03,  2.72000000e+02,  1.80000000e+03,
        1.27005200e+03,  

In [482]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   day5.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   day4.ipynb
	modified:   matrix_one/day3.ipynb



In [0]:
!git add day5.ipynb

In [483]:
!git commit -m "Add notebook from day5"


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@ab9d09e67d10.(none)')


In [0]:
!git config --global user.email "fnoworolnik@gmail.com"
!git config --global user.name "Filip Noworolnik"